# Programming for Data Analysis - Project 2


Order of Business:
- Join CO2 datasets into single pandas dataframe spanning 800K years to present.
- Join temperature datasets into single pandas dataframe spanning 800K years to present.
- Investigate Jouzel's temperature anomoly calculations.
- Join datasets for one other feature (CH4 or polar ice coverage) into single pandas dataframe spanning 800K years to present.
- Join datsets for temperature and rainfall in Ireland into single pandas dataframe.
- Join all above datsets into single pandas dataframe.
- Export datsets to csv and json formats.
- Graph CO2 vs temperature vs additional feature vs Irish data.
- Analyse the above for correlations, lead/lag, etc.
- Use linear regression to predict temperature change over coming decades.
- Look at recent data and analyse to see if recent increases have been linear or nonlinear.




Most recent CO2 data:
Lan, X., Tans, P. and K.W. Thoning: Trends in globally-averaged CO2 determined from NOAA Global Monitoring Laboratory measurements. Version 2023-12 https://doi.org/10.15138/9N0H-ZH07


In [2]:
# Import packages useful for data importing and analysis

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
# Create a pandas datframe with IPCC CO2 data.

co2_ipcc = pd.read_csv('data/co2_ipcc_clean.csv')
co2_ipcc

,Gasage (yr BP),CO2 (ppmv),sigma mean CO2 (ppmv)
0,-51.03,368.02,0.06
1,-48.00,361.78,0.37
2,-46.28,359.65,0.10
3,-44.41,357.11,0.16
4,-43.08,353.95,0.04
...,...,...,...
1895,803709.25,202.65,1.04
1896,803925.28,202.92,2.06
1897,804009.87,207.50,0.92
1898,804522.67,204.86,1.64


In [8]:
# Create a pandas dataframe with NOAA data.

co2_noaa = pd.read_csv('data/co2_annmean_gl.csv', skiprows=37)
co2_noaa

,year,mean,unc
0,1979,336.85,0.11
1,1980,338.91,0.07
2,1981,340.11,0.09
3,1982,340.85,0.03
4,1983,342.53,0.06
5,1984,344.07,0.08
6,1985,345.54,0.07
7,1986,346.97,0.07
8,1987,348.68,0.10
9,1988,351.16,0.07
